Recuperation des donnes on filtre sur les donnees qui sont utiles celle qui correspondent à des tickets de vente

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
d = pd.read_csv('data2.csv', sep=',')

d=d.loc[d['Quantity'] > 0]
d=d.loc[d['Description'] != "?"]
d=d.loc[d['Description'] != "DOTCOM POSTAGE"]
data = d.iloc[:,[0,1]]
vente=d.iloc[:,[2,3]]

Fonction utilitaire:
Permet de passer de nom a code et inversement

In [5]:
produits = d.iloc[:,[1,2]].drop_duplicates(subset='StockCode')
produits2 = d.iloc[:,[1,2]].drop_duplicates(subset='Description')
def codeToName(code):
    result= produits.loc[produits['StockCode'] == code]
    if(result.size>0):
        return result.Description.values[0]
    else:
        return "ProduitNonTrouve"
def nameToCode(name):
    result= produits2.loc[produits2['Description'] == name]
    if(result.size>0):
        return result.StockCode.values[0]
    else:
        return "ProduitNonTrouve"

on transforme la base de donne en liste de transaction

In [6]:
groupe = data.groupby('InvoiceNo')
transaction=[]
transactions=[]
for name,group in groupe:
    for x in group.StockCode:
        transaction.append(x)
    transactions.append(transaction)
    transaction=[]
    

Bibliotheque implementant apriori

In [7]:
from apyori import apriori  

ici on parametre l'algo pour avoir un nombre de regles d'associations suffisantes tout en ayant un temps de calcul limité:
7819 associations
~30 min de temps de calcul
<h3> attention avant de lancer ce bout de code ca prend du temps </H3>

In [146]:
association_rules = apriori(transactions, min_support=0.005, min_confidence=0.5, min_lift=1, min_length=2)  
association_results = list(association_rules) 
print(len(association_results)) 

7819


fonction qui permet de connaitre ce qui va etre achete avec un produit
amelioration possible:
avec plusieurs items

In [24]:
def acheteAvec(assoc, base):
    listeRetour = []
    confiance=0
    elem=["codeItem",confiance]
    print("avec: " + codeToName(base)+"\nIl va acheter:\n")
    for relations in assoc:
        for relation in relations[2]:
            if base in relation[0] and len(relation[0])==1:
                for x in relation[1]:
                    elem.append(x)
                    print(codeToName(x))
                print("avec une confiance de: ")
                elem.append(relation[2])
                print(relation[2])
                listeRetour.append(elem)
                elem.clear()
                print()
    return listeRetour
        

test

In [9]:
acheteAvec(association_results,nameToCode("WHITE HANGING HEART T-LIGHT HOLDER"))


NameError: name 'association_results' is not defined

Test: pour faire que du apriori sur les transactions contenant notre item

on recupere seulement les tickets contenant notre item

In [10]:
def contenant(transactions, item):
    TransactionsRetour=[]
    for motif in transactions:
        if item in motif:
            TransactionsRetour.append(motif)
    return TransactionsRetour

In [11]:
TransactionsItem = contenant(transactions,"22727")
len(TransactionsItem)

1051

Puis on fait un apriori dessus
Il faut chercher a ameliorer le calcul de la confidence

In [12]:
association_rules = apriori(TransactionsItem, min_support=0.2, min_confidence=0.2, min_lift=0, min_length=2)
association_results = list(association_rules) 
print(len(association_results)) 

23


In [32]:
def faitTout(code):
    TransactionsItem = contenant(transactions,code)
    print(len(TransactionsItem))
    if(len(TransactionsItem)>1000):
        sup=0.1
    else:
        sup=1/len(TransactionsItem)*100
    if(sup>0.3):
        sup=0.3
    association_rules = apriori(TransactionsItem, min_support=sup, min_confidence=0.2, min_lift=0, min_length=2)
    association_results = list(association_rules)
    acheteAvec(association_results,code)

SyntaxError: invalid syntax (<ipython-input-32-8f9f456a62a0>, line 4)

In [33]:
faitTout(["22689",nameToCode("PAPER CHAIN KIT 50'S CHRISTMAS")])

ZeroDivisionError: division by zero